In [1]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install accelerate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install bitsandbytes

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch
print(torch.cuda.is_available())

True


In [6]:
%pip install huggingface_hub


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
%pip install --upgrade bitsandbytes

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
%pip install --upgrade transformers torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Oct_30_01:18:48_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0


In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "microsoft/Phi-3.5-mini-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype = torch.float16
).to(device)

device = torch.device("cuda:0")

# Get allocated, reserved, and total memory on this GPU
allocated_memory = torch.cuda.memory_allocated(device)
reserved_memory = torch.cuda.memory_reserved(device)
max_memory = torch.cuda.get_device_properties(device).total_memory

print(f"Allocated: {allocated_memory / (1024**2):.2f} MB")
print(f"Reserved:  {reserved_memory / (1024**2):.2f} MB")
print(f"Total:     {max_memory / (1024**2):.2f} MB")

print(model.device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.03s/it]


Allocated: 7296.53 MB
Reserved:  7312.00 MB
Total:     8187.50 MB
cuda:0


In [59]:
prompt = "How to cook a birthday cake?"
inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)  # unload explicit ?
# outputs = model.generate(**inputs, max_new_tokens=500)
outputs = model.generate(
    **inputs,
    max_new_tokens=2000,      # Generate up to 500 new tokens
    min_length=200,           # Ensure at least 200 tokens are generated
    temperature=0.6,
    top_p=0.9,                # Consider the top 90% of probabilities
    num_beams=1,              # Use greedy decoding for speed
    do_sample=True,           # Enable sampling
    use_cache=True,           # Use cached key/values for faster generation
    early_stopping=False      # num_beams 1 forced
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# 40s vs 2m40s

How to cook a birthday cake?

To cook a birthday cake, you will need to follow these steps:

1. Gather ingredients: You will need flour, sugar, eggs, butter, baking powder, milk, and flavorings such as vanilla extract or cocoa powder.

2. Preheat the oven: Preheat your oven to the appropriate temperature for the recipe you are using.

3. Prepare the cake batter: In a mixing bowl, combine the dry ingredients (flour, sugar, and baking powder) and mix well. In a separate bowl, beat together the wet ingredients (eggs, butter, and milk). Gradually add the wet ingredients to the dry ingredients and mix until the batter is smooth.

4. Add flavorings: If you want to add flavorings such as vanilla extract or cocoa powder, add them to the batter now and mix well.

5. Prepare the cake pan: Grease and flour a cake pan or line it with parchment paper.

6. Pour the batter into the pan: Pour the cake batter into the prepared cake pan, smoothing the top with a spatula.

7. Bake the cake: Place the cak

In [21]:
max_length = model.config.n_positions if hasattr(model.config, "n_positions") else model.config.max_position_embeddings
print(max_length)

131072


In [53]:
base_model_dir = "./phi-3.5-instruction"
model.save_pretrained(base_model_dir)
tokenizer.save_pretrained(base_model_dir)

('./phi-3.5-onnx\\tokenizer_config.json',
 './phi-3.5-onnx\\special_tokens_map.json',
 './phi-3.5-onnx\\tokenizer.json')

In [85]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_dir,
    torch_dtype = torch.float16
).to(device)

tokenizer = AutoTokenizer.from_pretrained(base_model_dir)

device = torch.device("cuda:0")

# Get allocated, reserved, and total memory on this GPU
allocated_memory = torch.cuda.memory_allocated(device)
reserved_memory = torch.cuda.memory_reserved(device)
max_memory = torch.cuda.get_device_properties(device).total_memory

print(f"Allocated: {allocated_memory / (1024**2):.2f} MB")
print(f"Reserved:  {reserved_memory / (1024**2):.2f} MB")
print(f"Total:     {max_memory / (1024**2):.2f} MB")

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  6.64it/s]


Allocated: 7296.53 MB
Reserved:  8324.00 MB
Total:     8187.50 MB


In [87]:
prompt = "Cake recipe and instructions"
inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)  # unload explicit ?
# outputs = model.generate(**inputs, max_new_tokens=500)
outputs = model.generate(
    **inputs,
    max_new_tokens=500,
    min_length=0,
    temperature=0.6,
    top_p=0.8,
    num_beams=3,
    length_penalty=0.8,  # kaznjava duzinu teksta srednje strogo
    do_sample=True,
    use_cache=True,
    early_stopping=True   
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
del inputs
torch.cuda.empty_cache()


Cake recipe and instructions

Ingredients:

- 2 cups all-purpose flour
- 1 1/2 cups granulated sugar
- 3/4 cup unsweetened cocoa powder
- 1 1/2 teaspoons baking powder
- 1 1/2 teaspoons baking soda
- 1 teaspoon salt
- 2 large eggs
- 1 cup whole milk
- 1/2 cup vegetable oil
- 2 teaspoons pure vanilla extract
- 1 cup boiling water

Instructions:

1. Preheat your oven to 350°F (175°C). Grease and flour two 9-inch round cake pans.

2. In a large mixing bowl, whisk together the flour, sugar, cocoa powder, baking powder, baking soda, and salt.

3. Add the eggs, milk, vegetable oil, and vanilla extract to the dry ingredients. Beat on medium speed for about 2 minutes until the batter is smooth and well combined.

4. Gradually add the boiling water to the batter, mixing continuously. The batter will be thin, but that's okay.

5. Pour the batter evenly into the prepared cake pans.

6. Bake for 30-35 minutes, or until a toothpick inserted into the center of the cakes comes out clean.

7. Remove t

In [91]:
allocated_memory = torch.cuda.memory_allocated(device)
reserved_memory = torch.cuda.memory_reserved(device)
max_memory = torch.cuda.get_device_properties(device).total_memory

print(f"Allocated: {allocated_memory / (1024**2):.2f} MB")
print(f"Reserved:  {reserved_memory / (1024**2):.2f} MB")
print(f"Total:     {max_memory / (1024**2):.2f} MB")

Allocated: 8.13 MB
Reserved:  22.00 MB
Total:     8187.50 MB


In [89]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [90]:
del model
del tokenizer
torch.cuda.empty_cache()
torch.cuda.synchronize()